In [1]:
# Example of LSTM to learn a sequence
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation
from keras.layers import LSTM
import matplotlib.pyplot as plt
import matplotlib
import time
import pandas as pd
import numpy as np
%matplotlib qt

Using TensorFlow backend.


In [ ]:
sin = 'sinwave.csv'
df = pd.read_csv(sin,header=0,squeeze=True)
# create X/y pairs
df = concat([df.shift(1), df], axis=1)
df.dropna(inplace=True)
# convert to LSTM friendly format
values = df.values
# split data into train and test
train_size = int(len(df) * 0.67)
test_size = len(df) - train_size
X,Y = values[:,0],values[:,1]
# train, test = series[0:train_size], series[train_size:len(series)]
trainX, trainY = X[0:train_size], Y[0:train_size]
testX, testY = X[-test_size:len(values)], Y[-test_size:len(values)][:-93]
trainX,testX = trainX.reshape(len(trainX), 1, 1), testX[:-93].reshape(len(testX[:-93]), 1, 1)
shifttestY = Y[-test_size:len(values)]

In [ ]:
# plt.figure()
# testX = testX[:-93,0].reshape(len(testX[:-93,0]), 1, 1)
# testY = testY[:-93]
plt.plot(testY)
print len(testY)

In [4]:
# reader for jakobs
seq_len = 1
def load_data(filename, seq_len, normalise_window):
    f = open(filename, 'rb').read()
    data = f.decode().split('\n')

    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    
    if normalise_window:
        result = normalise_windows(result)

    result = np.array(result)

    row = round(0.9 * result.shape[0])
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    return [x_train, y_train, x_test, y_test]

def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data
# X_train, y_train, X_test, y_test = load_data('sp500.csv', seq_len, True)
trainX, trainY, testX, testY = load_data('sp500.csv', seq_len, True)
shifttestY = testY[:-10]

In [ ]:
# create sequence
length = 1000
sequence = [i/float(length) for i in range(length)]
# print(sequence)
# create X/y pairs
df = DataFrame(sequence)
df = concat([df.shift(1), df], axis=1)
df.dropna(inplace=True)
# convert to LSTM friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
trainX = X.reshape(len(X), 1, 1)
trainY = y
testX = trainX
testY = y
# print trainX
# print trainX.shape
print "Done with data preparation"

In [5]:
# def plot_results(predicted_data, true_data):
#     fig = plt.figure(facecolor='white')
#     ax = fig.add_subplot(111)
#     ax.plot(true_data, label='True Data')
#     plt.plot(predicted_data, label='Prediction')
#     plt.legend()
#     plt.show()
models = {}
# trainX = X_train
# trainY = y_train
# testX = X_test
# testY = y_test
for i in range(0,3):
    state = i
    if state == 0:
        print "Fitting stateless model..."
        d0 = time.time()
        # 1. define network STATELESS
        model = Sequential()
        model.add(LSTM(10, input_shape=(1,1)))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        # 2. compile network
        model.compile(optimizer='adam', loss='mean_squared_error')
        # 3. fit network
        history = model.fit(trainX, trainY, epochs=1000, batch_size=len(trainX), verbose=1,validation_split=0.05)
        # summarize history for loss
        plt.figure()
        plt.plot(history.history['loss'],label='loss')
        plt.plot(history.history['val_loss'],label='val_loss')
        plt.title('model loss stateless')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend()
        plt.show()
        # 4. evaluate network
        loss = model.evaluate(testX, testY, verbose=0)
        print(loss)
        models['stateless'] = model
        d1 = time.time()
        dt = d1-d0
        print "Time fitting stateless: {}s".format(dt)
    elif state == 1:
        print "Fitting stateful model..."
        epoch = 20
        d0 = time.time()
        # 1. define network STATEFUL
        batch_size = 1
        model = Sequential()
        model.add(LSTM(10,batch_input_shape=(batch_size, 1, 1), stateful=True))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        # 2. compile network
        model.compile(optimizer='adam', loss='mean_squared_error')
        # 3. fit network
        loss = []
        val_loss = []
        for i in range(epoch):
            print "{}/{} epoch".format(i+1,epoch)
            history = model.fit(trainX, trainY, epochs=1, batch_size=batch_size,verbose=2,shuffle=False,validation_split=0.05)
            loss.append(history.history['loss'])
            val_loss.append(history.history['val_loss'])
            model.reset_states()
        # summarize history for loss
        plt.figure()
        plt.plot(loss,label='loss')
        plt.plot(val_loss,label='val_loss')
        plt.title('model loss stateful')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend()
        plt.show()
        models['stateful'] = model
        d1 = time.time()
        dt = d1-d0
        print "Time fitting stateless: {}s".format(dt)
    elif state == 2:
        print "Fitting jakobs model..."
        epoch = 20
#         trainX = X_train
#         trainY = y_train
        d0 = time.time()
        # Use jakobs model
        model = Sequential()
        # add hidden lstm with 50 neurons
        model.add(LSTM(
            50,
            input_shape = (1,1),
            return_sequences = True))
        model.add(Dropout(0.2))
        # add another lstm with 100 neurons
        model.add(LSTM(
            100,
            return_sequences=False))
        model.add(Dropout(0.2))
        # output to 1 neuron
        model.add(Dense(1))
        model.add(Activation("linear"))
        # compile model
        model.compile(loss='mse',optimizer='rmsprop')
        # fit model
        history = model.fit(trainX, trainY, batch_size=32, epochs=epoch, validation_split=0.05)
        # summarize history for loss
        plt.figure()
        plt.plot(history.history['loss'],label='loss')
        plt.plot(history.history['val_loss'],label='val_loss')
        plt.title('model loss jakobs')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend()
        plt.show()   
        # save to models
        models['jakob'] = model
        d1 = time.time()
        dt = d1-d0
        print "Time fitting jakob's: {}s".format(dt)

Fitting stateless model...
Train on 3564 samples, validate on 188 samples
Epoch 1/1000
3564/3564 [==============================] - 0s - loss: 1.6754e-04 - val_loss: 1.2028e-04
Epoch 2/1000
3564/3564 [==============================] - 0s - loss: 1.7008e-04 - val_loss: 1.1696e-04
Epoch 3/1000
3564/3564 [==============================] - 0s - loss: 1.6766e-04 - val_loss: 1.1651e-04
Epoch 4/1000
3564/3564 [==============================] - 0s - loss: 1.6807e-04 - val_loss: 1.1707e-04
Epoch 5/1000
3564/3564 [==============================] - 0s - loss: 1.6894e-04 - val_loss: 1.1674e-04
Epoch 6/1000
3564/3564 [==============================] - 0s - loss: 1.6843e-04 - val_loss: 1.1636e-04
Epoch 7/1000
3564/3564 [==============================] - 0s - loss: 1.6775e-04 - val_loss: 1.1669e-04
Epoch 8/1000
3564/3564 [==============================] - 0s - loss: 1.6757e-04 - val_loss: 1.1749e-04
Epoch 9/1000
3564/3564 [==============================] - 0s - loss: 1.6801e-04 - val_loss: 1.1792e-04

3564/3564 [==============================] - 0s - loss: 1.6751e-04 - val_loss: 1.1656e-04
Epoch 80/1000
3564/3564 [==============================] - 0s - loss: 1.6755e-04 - val_loss: 1.1654e-04
Epoch 81/1000
3564/3564 [==============================] - 0s - loss: 1.6752e-04 - val_loss: 1.1651e-04
Epoch 82/1000
3564/3564 [==============================] - 0s - loss: 1.6751e-04 - val_loss: 1.1646e-04
Epoch 83/1000
3564/3564 [==============================] - 0s - loss: 1.6748e-04 - val_loss: 1.1644e-04
Epoch 84/1000
3564/3564 [==============================] - 0s - loss: 1.6749e-04 - val_loss: 1.1646e-04
Epoch 85/1000
3564/3564 [==============================] - 0s - loss: 1.6750e-04 - val_loss: 1.1648e-04
Epoch 86/1000
3564/3564 [==============================] - 0s - loss: 1.6746e-04 - val_loss: 1.1653e-04
Epoch 87/1000
3564/3564 [==============================] - 0s - loss: 1.6755e-04 - val_loss: 1.1655e-04
Epoch 88/1000
3564/3564 [==============================] - 0s - loss: 1.6755e-

3564/3564 [==============================] - 0s - loss: 1.6753e-04 - val_loss: 1.1652e-04
Epoch 158/1000
3564/3564 [==============================] - 0s - loss: 1.6753e-04 - val_loss: 1.1653e-04
Epoch 159/1000
3564/3564 [==============================] - 0s - loss: 1.6751e-04 - val_loss: 1.1652e-04
Epoch 160/1000
3564/3564 [==============================] - 0s - loss: 1.6750e-04 - val_loss: 1.1649e-04
Epoch 161/1000
3564/3564 [==============================] - 0s - loss: 1.6756e-04 - val_loss: 1.1648e-04
Epoch 162/1000
3564/3564 [==============================] - 0s - loss: 1.6758e-04 - val_loss: 1.1649e-04
Epoch 163/1000
3564/3564 [==============================] - 0s - loss: 1.6748e-04 - val_loss: 1.1652e-04
Epoch 164/1000
3564/3564 [==============================] - 0s - loss: 1.6742e-04 - val_loss: 1.1652e-04
Epoch 165/1000
3564/3564 [==============================] - 0s - loss: 1.6757e-04 - val_loss: 1.1651e-04
Epoch 166/1000
3564/3564 [==============================] - 0s - loss:

3564/3564 [==============================] - 0s - loss: 1.6753e-04 - val_loss: 1.1649e-04
Epoch 236/1000
3564/3564 [==============================] - 0s - loss: 1.6743e-04 - val_loss: 1.1650e-04
Epoch 237/1000
3564/3564 [==============================] - 0s - loss: 1.6758e-04 - val_loss: 1.1650e-04
Epoch 238/1000
3564/3564 [==============================] - 0s - loss: 1.6750e-04 - val_loss: 1.1650e-04
Epoch 239/1000
3564/3564 [==============================] - 0s - loss: 1.6750e-04 - val_loss: 1.1648e-04
Epoch 240/1000
3564/3564 [==============================] - 0s - loss: 1.6746e-04 - val_loss: 1.1648e-04
Epoch 241/1000
3564/3564 [==============================] - 0s - loss: 1.6755e-04 - val_loss: 1.1650e-04
Epoch 242/1000
3564/3564 [==============================] - 0s - loss: 1.6759e-04 - val_loss: 1.1652e-04
Epoch 243/1000
3564/3564 [==============================] - 0s - loss: 1.6749e-04 - val_loss: 1.1652e-04
Epoch 244/1000
3564/3564 [==============================] - 0s - loss:

3564/3564 [==============================] - 0s - loss: 1.6756e-04 - val_loss: 1.1651e-04
Epoch 314/1000
3564/3564 [==============================] - 0s - loss: 1.6749e-04 - val_loss: 1.1651e-04
Epoch 315/1000
3564/3564 [==============================] - 0s - loss: 1.6741e-04 - val_loss: 1.1649e-04
Epoch 316/1000
3564/3564 [==============================] - 0s - loss: 1.6747e-04 - val_loss: 1.1648e-04
Epoch 317/1000
3564/3564 [==============================] - 0s - loss: 1.6745e-04 - val_loss: 1.1649e-04
Epoch 318/1000
3564/3564 [==============================] - 0s - loss: 1.6753e-04 - val_loss: 1.1651e-04
Epoch 319/1000
3564/3564 [==============================] - 0s - loss: 1.6751e-04 - val_loss: 1.1653e-04
Epoch 320/1000
3564/3564 [==============================] - 0s - loss: 1.6756e-04 - val_loss: 1.1651e-04
Epoch 321/1000
3564/3564 [==============================] - 0s - loss: 1.6750e-04 - val_loss: 1.1648e-04
Epoch 322/1000
3564/3564 [==============================] - 0s - loss:

...

3564/3564 [==============================] - 0s - loss: 1.6763e-04 - val_loss: 1.1650e-04
Epoch 704/1000
3564/3564 [==============================] - 0s - loss: 1.6754e-04 - val_loss: 1.1649e-04
Epoch 705/1000
3564/3564 [==============================] - 0s - loss: 1.6755e-04 - val_loss: 1.1650e-04
Epoch 706/1000
3564/3564 [==============================] - 0s - loss: 1.6753e-04 - val_loss: 1.1651e-04
Epoch 707/1000
3564/3564 [==============================] - 0s - loss: 1.6755e-04 - val_loss: 1.1651e-04
Epoch 708/1000
3564/3564 [==============================] - 0s - loss: 1.6750e-04 - val_loss: 1.1649e-04
Epoch 709/1000
3564/3564 [==============================] - 0s - loss: 1.6758e-04 - val_loss: 1.1649e-04
Epoch 710/1000
3564/3564 [==============================] - 0s - loss: 1.6748e-04 - val_loss: 1.1649e-04
Epoch 711/1000
3564/3564 [==============================] - 0s - loss: 1.6749e-04 - val_loss: 1.1648e-04
Epoch 712/1000
3564/3564 [==============================] - 0s - loss:

3564/3564 [==============================] - 0s - loss: 1.6746e-04 - val_loss: 1.1648e-04
Epoch 782/1000
3564/3564 [==============================] - 0s - loss: 1.6752e-04 - val_loss: 1.1655e-04
Epoch 783/1000
3564/3564 [==============================] - 0s - loss: 1.6750e-04 - val_loss: 1.1654e-04
Epoch 784/1000
3564/3564 [==============================] - 0s - loss: 1.6754e-04 - val_loss: 1.1649e-04
Epoch 785/1000
3564/3564 [==============================] - 0s - loss: 1.6750e-04 - val_loss: 1.1647e-04
Epoch 786/1000
3564/3564 [==============================] - 0s - loss: 1.6751e-04 - val_loss: 1.1649e-04
Epoch 787/1000
3564/3564 [==============================] - 0s - loss: 1.6751e-04 - val_loss: 1.1652e-04
Epoch 788/1000
3564/3564 [==============================] - 0s - loss: 1.6750e-04 - val_loss: 1.1652e-04
Epoch 789/1000
3564/3564 [==============================] - 0s - loss: 1.6755e-04 - val_loss: 1.1650e-04
Epoch 790/1000
3564/3564 [==============================] - 0s - loss:

3564/3564 [==============================] - 0s - loss: 1.6750e-04 - val_loss: 1.1650e-04
Epoch 860/1000
3564/3564 [==============================] - 0s - loss: 1.6754e-04 - val_loss: 1.1648e-04
Epoch 861/1000
3564/3564 [==============================] - 0s - loss: 1.6745e-04 - val_loss: 1.1650e-04
Epoch 862/1000
3564/3564 [==============================] - 0s - loss: 1.6755e-04 - val_loss: 1.1650e-04
Epoch 863/1000
3564/3564 [==============================] - 0s - loss: 1.6741e-04 - val_loss: 1.1648e-04
Epoch 864/1000
3564/3564 [==============================] - 0s - loss: 1.6765e-04 - val_loss: 1.1649e-04
Epoch 865/1000
3564/3564 [==============================] - 0s - loss: 1.6762e-04 - val_loss: 1.1652e-04
Epoch 866/1000
3564/3564 [==============================] - 0s - loss: 1.6762e-04 - val_loss: 1.1652e-04
Epoch 867/1000
3564/3564 [==============================] - 0s - loss: 1.6751e-04 - val_loss: 1.1648e-04
Epoch 868/1000
3564/3564 [==============================] - 0s - loss:

3564/3564 [==============================] - 0s - loss: 1.6759e-04 - val_loss: 1.1652e-04
Epoch 938/1000
3564/3564 [==============================] - 0s - loss: 1.6760e-04 - val_loss: 1.1649e-04
Epoch 939/1000
3564/3564 [==============================] - 0s - loss: 1.6747e-04 - val_loss: 1.1648e-04
Epoch 940/1000
3564/3564 [==============================] - 0s - loss: 1.6755e-04 - val_loss: 1.1649e-04
Epoch 941/1000
3564/3564 [==============================] - 0s - loss: 1.6754e-04 - val_loss: 1.1653e-04
Epoch 942/1000
3564/3564 [==============================] - 0s - loss: 1.6744e-04 - val_loss: 1.1653e-04
Epoch 943/1000
3564/3564 [==============================] - 0s - loss: 1.6759e-04 - val_loss: 1.1649e-04
Epoch 944/1000
3564/3564 [==============================] - 0s - loss: 1.6750e-04 - val_loss: 1.1645e-04
Epoch 945/1000
3564/3564 [==============================] - 0s - loss: 1.6753e-04 - val_loss: 1.1649e-04
Epoch 946/1000
3564/3564 [==============================] - 0s - loss:

5s - loss: 1.7302e-04 - val_loss: 1.1753e-04
14/20 epoch
Train on 3564 samples, validate on 188 samples
Epoch 1/1
5s - loss: 1.7299e-04 - val_loss: 1.1753e-04
15/20 epoch
Train on 3564 samples, validate on 188 samples
Epoch 1/1
5s - loss: 1.7300e-04 - val_loss: 1.1755e-04
16/20 epoch
Train on 3564 samples, validate on 188 samples
Epoch 1/1
5s - loss: 1.7304e-04 - val_loss: 1.1760e-04
17/20 epoch
Train on 3564 samples, validate on 188 samples
Epoch 1/1
5s - loss: 1.7305e-04 - val_loss: 1.1753e-04
18/20 epoch
Train on 3564 samples, validate on 188 samples
Epoch 1/1
5s - loss: 1.7297e-04 - val_loss: 1.1753e-04
19/20 epoch
Train on 3564 samples, validate on 188 samples
Epoch 1/1
5s - loss: 1.7298e-04 - val_loss: 1.1749e-04
20/20 epoch
Train on 3564 samples, validate on 188 samples
Epoch 1/1
5s - loss: 1.7298e-04 - val_loss: 1.1753e-04
Time fitting stateless: 111.166321993s
Fitting jakobs model...
Train on 3564 samples, validate on 188 samples
Epoch 1/20
3564/3564 [=========================

In [ ]:
# # Try new data pred
# import numpy as np
# length = 50
# sequence = np.arange(0.999,1.01,0.001)
# X = sequence
# X = X.reshape(len(X), 1, 1)
# testX = testX[-50:]
# testY = testY[-50:]
history.history

In [ ]:
# 5. make predictions
from sklearn.metrics import mean_squared_error
from math import sqrt
for mod in models.keys():
    if mod == 'stateful':
        predictions = models[mod].predict(testX,batch_size=1, verbose=0)
        rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure(0)
        print "Stateful prediction..."
        print testX[-1],testY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Stateful')
        plt.plot(testX[:, 0],label='initial')
        plt.plot(testY,label='shifted true')
        plt.plot(predictions,label='shifted pred')
        plt.legend()
        plt.show()
    elif mod == 'stateless':
        predictions = models[mod].predict(testX, verbose=0)
        rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure(1)
        print "Stateless prediction..."
        print testX[-1],testY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Stateless')
        plt.plot(testX[:, 0],label='initial')
        plt.plot(testY,label='shifted true')
        plt.plot(predictions,label='shifted pred')
        plt.legend()
        plt.show()
    elif mod == 'jakob':
#         testX = X_test
#         testY = y_test
        predictions = models[mod].predict(testX)
#         predictions = np.reshape(predictions, (predictions.size,))
        rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure(3)
        print "Jakob's prediction..."
        print testX[-1],testY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Jakobs')
        plt.plot(testX[:, 0],label='initial')
        plt.plot(testY,label='shifted true')
        plt.plot(predictions,label='shifted pred')
        plt.legend()
        plt.show()
# stateful = True
# if stateful:
#     predictions = model.predict(X,batch_size=1, verbose=0)
# else:
#     predictions = model.predict(X, verbose=0)
# # print(predictions[:, 0][-10:])
# # predictions = predictions[-10:]
# print X[-1],y[-1],predictions[-1][0]
# plt.plot(X[:, 0])
# plt.plot(predictions)

In [7]:
# 5. make predictions but shift forward n steps
from sklearn.metrics import mean_squared_error
from math import sqrt
n = 10
for mod in models.keys():
    if mod == 'stateful':
        predicted = testX[:,0]
        for i in range(n):
            predictions = models[mod].predict(predicted.reshape(len(predicted),1,1),batch_size=1, verbose=0)
            predicted = np.append(predicted,predictions[-1])
            rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure()
        print "Stateful prediction..."
        print testX[-1],testY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Stateful')
        plt.plot(testX[:, 0],label='initial')
        plt.plot(shifttestY,label='shifted true')
        plt.plot(predicted,label='shifted pred')
        plt.legend()
        plt.show()
    elif mod == 'stateless':
        predicted = testX[:,0]
        for i in range(n):
            predictions = models[mod].predict(predicted.reshape(len(predicted),1,1), verbose=0)
            predicted = np.append(predicted,predictions[-1])
            rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure()
        print "Stateless prediction..."
        print testX[-1],testY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Stateless')
        plt.plot(testX[:, 0],label='initial')
        plt.plot(shifttestY,label='shifted true')
        plt.plot(predicted,label='shifted pred')
        plt.legend()
        plt.show()
    elif mod == 'jakob':
        predicted = testX[:,0]
        # testX = X_test
        # testY = y_test
        for i in range(n):
            predictions = models[mod].predict(predicted.reshape(len(predicted),1,1))
#             print predictions[-1]
            predicted = np.append(predicted,predictions[-1])
            rmse = sqrt(mean_squared_error([testY[-1]],predictions[-1]))
        plt.figure()
        print "Jakob's prediction..."
        print testX[-1],testY[-1],predictions[-1][0]
        print "RMSE: {}".format(rmse)
        plt.title('Jakobs')
        plt.plot(testX[:, 0],label='initial')
        plt.plot(shifttestY,label='shifted true')
        plt.plot(predicted,label='shifted pred')
        plt.legend()
        plt.show()

Stateful prediction...
[[ 0.]] 0.0016313092282 0.000904393
RMSE: 0.000726916253285
Stateless prediction...
[[ 0.]] 0.0016313092282 0.00019815
RMSE: 0.00143315911637
Jakob's prediction...
[[ 0.]] 0.0016313092282 -0.00565137
RMSE: 0.00728267519381


In [8]:
len(predicted)

427

Stateful prediction...50
[[ 0.80588467]] 0.841470985 0.828496
RMSE: 0.0129754229234
Stateless prediction...10
[[ 0.80588467]] 0.841470985 0.782955
RMSE: 0.0585164113687

In [ ]:
testX[:,0]

In [ ]:
plt.plot(testY,label='shifted pred')

In [ ]:
from keras.utils import plot_model
plot_model(models['stateful'], to_file='model_stateful.png',show_shapes=True)

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(models['stateful']).create(prog='dot', format='svg'))